In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Desktop\\Text-Summarization-NLP-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Admin\\Desktop\\Text-Summarization-NLP-Project'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from textSummarizer.constants import *  
from textSummarizer.utils.common import create_directories, read_yaml

In [1]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config    

NameError: name 'CONFIG_FILE_PATH' is not defined

In [30]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [37]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
         
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [38]:
from src.textSummarizer.exceptions import CustomException
import sys

In [41]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e, sys)

[2023-10-05 21:19:01,761: INFO: common: yaml file: config\config.yml loaded successfully]
[2023-10-05 21:19:01,764: INFO: common: yaml file: params.yml loaded successfully]
[2023-10-05 21:19:01,767: INFO: common: created directory at: artifacts]
[2023-10-05 21:19:01,768: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-05 21:19:21,236: INFO: 1707712694: artifacts/data_ingestion/samsumdata.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7D84:35BE:5FEE0C:758A86:651F1577
Accept-Ranges: bytes
Date: Thu, 05 Oct 2023 20:20:01 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10541-MRS
X-Cache